# Working with time series, frequency series, and unit conversion

In this notebook, we show some of the most useful features of the :py:mod:`~timeseries` module. To do so, we will analyze a fake gravitational-wave signal. We will also show the :py:mod:`~frequencyseries` module and the :py:mod:`~unitconv` modules.

First, let's generate this signal. 

(This notebook is meant to be converted in Sphinx documentation and not used directly.)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from postcactus import timeseries as ts
from postcactus import unitconv as uc
from postcactus.gw_utils import luminosity_distance_to_redshift

%matplotlib inline

In [ ]:
t = np.linspace(0, 20, 500)
y = np.sin(t)

# Generate a TimeSeries by providing the times and the values of the series
gw = ts.TimeSeries(t, y)

To access the times and the values, use ``gw.t`` and ``gw.y``. 

In [ ]:
def plot(ser, *args, **kwargs):
    """Plot ser with labels"""
    plt.ylabel("d h")
    plt.xlabel("t")
    plt.plot(ser.t, ser.y, *args, **kwargs)
    
plot(gw)

Let's make this more interesting: we square the signal and add Gaussian noise. :py:class:`TimeSeries` support directly using all the basic mathematical operations in Python and NumPy, so we can easily square the signal and add another TimeSeries to it.

In [ ]:
noise = np.random.normal(0, 0.1, len(gw))  # Here we used len() on the TimeSeries
noise_ts = ts.TimeSeries(t, noise)

gw += noise_ts
gw **= 2

plot(gw)

To analyze this signal, first we will smooth it. To do that, we will use the :py:met:`~savgol_smooth_time` method, which is a generalized "running average" filter. This method requires length of time over which we want to smooth the data.

In :py:class:`~TimeSeries` there are always two different methods to do the same task, one with imperative verb (e.g., smooth), and the other with the past tense (e.g., smoothed). The first modifies the data, the second returns a new `TimeSeries` with the operation applied. Here, we will find what smoothing length to use by trial and error, so we will use the second method. 

In [ ]:
tsmooth = np.linspace(1, 3, 5)

plt.ylabel('y')
plt.xlabel('t')
for tsm in tsmooth:
    smoothed = gw.savgol_smoothed_time(tsm)
    plt.plot(smoothed.t, smoothed.y, label=tsm)
plt.legend()

From visual inspection it looks like that ``tsmooth = 1.5`` will work yield a clean series faithful to the original one.

In [ ]:
gw.savgol_smooth_time(1.5)
plot(gw)

Next, for fun, we can compute integrals and derivatives. For instance, we can compute what is the integral from 5 to 10.

In [ ]:
gw_int = gw.integrated()
a = 5
b = 10
print(f"The integral from {a} to {b} is {gw_int(b) - gw_int(a):.4f}")
plot(gw_int)

Here, we showed a very powerful feature of TimeSeries: you can call them on a specific time (as we did we ``gw_int(b)``). This is done using splines to interpolate to the points that are not available. Splines can also be used to take derivatives. Alternatively, one can simply take the finite (central) difference. Let's see what's the derivative of ``gw`` using splines and finite difference.

In [ ]:
gw_spline_der = gw.spline_derived()
gw_numer_der = gw.derived()

plot(gw_spline_der, label='spline')
plot(gw_numer_der, label='finite')
plt.legend()

Clearly, derivatives will be noisier than the actual data, so often it is convenient to smooth them out as shown before.

TimeSeries support complex signals. Now, we will create one using `gw` itself. We will copy `gw`, time-shift it, find the common time interval with the original `gw`, and use that as a the imaginary part.

In [ ]:
gw_imag = gw.copy()  # It is important to deep copy the object
gw_imag.time_shift(4)

plot(gw_imag)

In [ ]:
gw, gw_imag = ts.sample_common([gw, gw_imag])  # Resampling to common times

plot(gw, label="orig")
plot(gw_imag, label="new")
plt.legend()

In [ ]:
gw_complex = ts.TimeSeries(gw.t, gw.y + 1j * gw_imag.y)

plot(gw_complex.real(), label="real")
plot(gw_complex.imag(), label="imag")
plt.legend()

We can also compute the phase and absolute value. In particular, we will compute the unfolded phase (no wrapping over 2 :math:`\pi`)

In [ ]:
plot(gw_complex.abs(), label='abs')
plot(gw_complex.unfolded_phase(), label='phase')
plt.legend()

Here, the unfolded phase looks a little bit unusual. This is because we made up the signal.

We can also compute the frequency of the phase, which we can directly smooth.

In [ ]:
plot(gw_complex.phase_frequency(tsmooth=1.5))

Next, we will take a Fourier transform. Before, let's pretend that the signal was in geometrized units (as in simulations), and let's make it physical assuming a scale of :math:`M = 1 M_\odot`. For that, use the :py:mod:`~unitconv` module. We define a CU object that knows how to convert units.

In [ ]:
CU = uc.geom_umass_msun(1) 

# How to convert from geometrized length to physical length?
# Simply multiply times CU.length. Let's check that it is 1.477 km
CU.length  # m

Now we rescale `gw`, assuming that `y` is strain times distance (as usually is). Let's assume a distance of 500 Mpc.

In [ ]:
d_Mpc = 500

# inverse = True means from geometrized to physical
gw_physical = gw_complex.time_unit_changed(CU.time, inverse=True)  

gw_physical *= CU.length  # dh -> dh physical
# Now just the strain, since we assume a distance
gw_physical /= (d_Mpc * uc.MEGAPARSEC_SI)

# We have to manually add the redshift
gw_physical.redshifted(luminosity_distance_to_redshift(d_Mpc))

plt.xlabel("t [s]")
plt.ylabel("h")
plt.plot(gw_physical.t, gw_physical.y)